# Train a hypergraph neural network using UniGCNII layers

This tutorial consists of three main steps:
1. Loading the CiCitationCora dataset and lifting it to the hypergraph domain.
2. Defining a hypergraph neural network (HGNN) which utlilizes the UniGCNII layer, and
3. Training the obtained neural network on the training data and evaluating it on test data.

First, we import the neccessary packages.


In [1]:
import torch
import pickle
import numpy as np
import scipy.sparse as sp

from topomodelx.nn.hypergraph.unigcnii_layer import UniGCNIILayer
from topomodelx.nn.hypergraph.unigcnii import UniGCNII

If GPUs are available, we want to make use of them, otherwise the model is run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Loading the data

We are using the Cora co-citation dataset. Here, the nodes represent documents and the edges in the graph represent documents which are co-cited. It is possible to compute this graph from the citation network directly. However, this is computationally very expensive. Instead, we load the dataset directly as it is available to download.

The task here is to classify each node and assign one of 7 possible classes to it. The dataset is a standard benchmark used in HGNN literature.

In [3]:
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/features.pickle
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/hypergraph.pickle
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/labels.pickle

--2023-07-13 14:18:50--  https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/features.pickle
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/features.pickle [following]
--2023-07-13 14:18:51--  https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/features.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404937 (395K) [application/octet-stream]
Saving to: ‘features.pickle’

features.pickle     100%[===================>] 395.45K  --.-KB

In [4]:
! wget https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/splits/1.pickle

--2023-07-13 14:18:53--  https://github.com/malllabiisc/HyperGCN/raw/master/data/cocitation/cora/splits/1.pickle
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/splits/1.pickle [following]
--2023-07-13 14:18:54--  https://raw.githubusercontent.com/malllabiisc/HyperGCN/master/data/cocitation/cora/splits/1.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51582 (50K) [application/octet-stream]
Saving to: ‘1.pickle’

1.pickle            100%[===================>]  50.37K  --.-KB/s    in 

Now, we can load the loaded dataset

In [5]:
with open("features.pickle", "rb") as handle:
    features = pickle.load(handle).todense()

with open("hypergraph.pickle", "rb") as handle:
    hypergraph = pickle.load(handle)

with open("labels.pickle", "rb") as handle:
    labels = pickle.load(handle)

/tmp/ipykernel_14655/121206761.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  features = pickle.load(handle).todense()


In [6]:
# transform the input and output features to pytorch
x_0 = sp.csr_matrix(np.array(features), dtype=np.float32)
x_0 = torch.FloatTensor(np.array(x_0.todense()))
x_0 = x_0.to(device)

y = torch.LongTensor(np.array(labels))
y = y.to(device)

# construct the incidence matrix
h = np.zeros((x_0.shape[0], len(hypergraph)))

for num, nodes in enumerate(hypergraph.values()):
    h[list(nodes), num] = 1

Finally, we add self-loops to the dataset i.e. for every node $v$, we add a hyper-edge only containing that specific node $e = \{ v \}$. This is the standard format expected by the GCNII layers and transform the matrix into a sparse pytorch tensor.

In [7]:
# add self loops
h2 = np.eye(x_0.shape[0])
incidence = np.hstack((h, h2))

# transform to pytorch
incidence = torch.Tensor(incidence).to_sparse_csr()
incidence = incidence.to(device)

/tmp/ipykernel_14655/2226475299.py:6: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)
  incidence = torch.Tensor(incidence).to_sparse_csr()


Now, we can load the predefine split in test and train data before we start constructing the HGNN.

In [8]:
# load the train-test split given by the dataset
with open("1.pickle", "rb") as H:
    splits = pickle.load(H)
    train, test = splits["train"], splits["test"]

train_idx = torch.LongTensor(train).to(device)
test_idx = torch.LongTensor(test).to(device)

## Training the neural network

First, we specify the hyperparameters of the training process.

In [10]:
num_epochs = 50
test_interval = 5

Next, we can generate the corresponding model, optimizer, and loss function with corresponding sizes.

In [11]:
channels = x_0.shape[1]
classes = 7  # current problem has 7 classes

In [12]:
model = UniGCNII(num_classes=classes, in_features=channels, num_layers=3).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

Now, we are ready to train the created model and evaluate the performance on the validation set.

In [13]:
for epoch in range(num_epochs):
    # set model to training mode
    model.train()

    y_hat = model(x_0, incidence)
    loss = loss_fn(y_hat[train_idx], y[train_idx])

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Evaluate performance on the validation data
    if (epoch + 1) % test_interval == 0:
        model.eval()
        with torch.no_grad():
            y_hat = model(x_0, incidence)
            y_pred = torch.argmax(y_hat, dim=1)

            train_acc = (y_pred[train_idx] == y[train_idx]).float().mean()
            test_acc = (y_pred[test_idx] == y[test_idx]).float().mean()

            print(
                f"Epoch: {epoch + 1} \t Train accuracy: {train_acc} \t Test accuracy: {test_acc}"
            )

Epoch: 5 	 Train accuracy: 0.7071428298950195 	 Test accuracy: 0.39291277527809143
Epoch: 10 	 Train accuracy: 0.75 	 Test accuracy: 0.4376946985721588
Epoch: 15 	 Train accuracy: 0.9357143044471741 	 Test accuracy: 0.6137071847915649
Epoch: 20 	 Train accuracy: 0.8714285492897034 	 Test accuracy: 0.45210281014442444
Epoch: 25 	 Train accuracy: 0.9071428775787354 	 Test accuracy: 0.5502336621284485
Epoch: 30 	 Train accuracy: 0.9785714149475098 	 Test accuracy: 0.5463395714759827
Epoch: 35 	 Train accuracy: 0.9785714149475098 	 Test accuracy: 0.5747663378715515
Epoch: 40 	 Train accuracy: 0.9857142567634583 	 Test accuracy: 0.5673676133155823
Epoch: 45 	 Train accuracy: 0.9428571462631226 	 Test accuracy: 0.552570104598999
Epoch: 50 	 Train accuracy: 0.9571428298950195 	 Test accuracy: 0.5607476830482483
